**Imports:**

In [1]:
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

import ipywidgets as widgets

# Plotting function

In [2]:
def find_indifference_curve(preferences='Cobb-Douglas',alpha=0.5,beta=0.5,x0=0.5,x1_max=10,x2_max=10,N=100):
            
    # a. select preferences
    if preferences == 'additive_crra' and np.isclose(alpha,1.0):
        preferences = 'additive_log'
        
    if preferences == 'cobb_douglas':
        
        utility_func = lambda x1,x2,alpha,beta: x1**alpha*x2**beta
        indiff_func_x1 = lambda u0,x1,alpha,beta: (u0/x1**alpha) ** (1/beta)
        indiff_func_x2 = lambda u0,x2,alpha,beta: (u0/x2**beta) ** (1/alpha)

    elif preferences == "ces":

        if beta == 0:

            utility_func = x1**alpha*x2**(1-alpha)
            indiff_func_x1 = (u0/x1**alpha)**(1/(1-alpha))
            indiff_func_x2 = (u0/x2*(1-alpha))**(1/alpha)

        else:
            utility_func = lambda x1,x2,alpha,beta: (alpha*x1**(-beta)+(1-alpha)*x2**(-beta))**(-1/beta)
            indiff_func_x1 = lambda u0,x1,alpha,beta: ((u0**(-beta)-alpha*x1**(-beta))/(1-alpha))**(-1/beta)
            indiff_func_x2 = lambda u0,x2,alpha,beta: ((u0**(-beta)-(1-alpha)*x2**(-beta))/alpha)**(-1/beta)

    elif preferences == "perfect_substitutes":

        utility_func = lambda x1,x2,alpha,beta: alpha*x1+beta*x2
        indiff_func_x1 = lambda u0,x1,alpha,beta: (u0-alpha*x1)/beta
        indiff_func_x2 = lambda u0,x2,alpha,beta: (u0-beta*x2)/alpha

    elif preferences == "quasi_sqrt":

        utility_func = lambda x1,x2,alpha,beta: alpha*np.sqrt(x1)+x2*beta
        indiff_func_x1 = lambda u0,x1,alpha,beta: (u0-alpha*np.sqrt(x1))/beta
        indiff_func_x2 = lambda u0,x2,alpha,beta: ((np.fmax(u0-x2*beta,0))/alpha)**2

    elif preferences == "quasi_log":

        utility_func = lambda x1,x2,alpha,beta: alpha*np.log(x1)+x2*beta
        indiff_func_x1 = lambda u0,x1,alpha,beta: (u0-alpha*np.log(x1))/beta
        indiff_func_x2 = lambda u0,x2,alpha,beta: np.exp((u0-x2*beta)/alpha)

    elif preferences == "additive_log":

        utility_func = lambda x1,x2,alpha,beta: np.log(x1)+beta*np.log(x2)
        indiff_func_x1 = lambda u0,x1,alpha,beta: np.exp((u0-np.log(x1))/beta)
        indiff_func_x2 = lambda u0,x2,alpha,beta: np.exp((u0-beta*np.log(x2)))

    elif preferences == "additive_crra":

        utility_func = lambda x1,x2,alpha,beta: x1**(1-alpha)/(1-alpha)+beta*x2**(1-alpha)/(1-alpha)
        indiff_func_x1 = lambda u0,x1,alpha,beta: (((1-alpha)*u0-x1**(1-alpha))/beta)**(1/(1-alpha))
        indiff_func_x2 = lambda u0,x2,alpha,beta: ((1-alpha)*u0-beta*x2**(1-alpha))**(1/(1-alpha))

    elif preferences == "concave":

        utility_func = lambda x1,x2,alpha,beta: alpha*x1**2+beta*x2**2
        indiff_func_x1 = lambda u0,x1,alpha,beta: np.sqrt((u0-alpha*x1**2)/beta)
        indiff_func_x2 = lambda u0,x2,alpha,beta: np.sqrt((u0-beta*x2**2)/alpha)

    else:

        raise ValueError(f'unknown preferences, {preferences}')

    u0 = utility_func(x0[0],x0[0],alpha,beta)
    
    # b. fix x1
    x1_x1 = np.linspace(0,x1_max,N)
    with np.errstate(divide="ignore", invalid="ignore"):
        x2_x1 = indiff_func_x1(u0,x1_x1,alpha,beta)

    # c. fix x2
    x2_x2 = np.linspace(0,x2_max,N)
    with np.errstate(divide="ignore", invalid="ignore"):
        x1_x2 = indiff_func_x2(u0,x2_x2, alpha, beta)

    # d. combine
    x1 = np.hstack([x1_x1, x1_x2])
    x2 = np.hstack([x2_x1, x2_x2])

    # e. clean
    with np.errstate(divide="ignore", invalid="ignore"):
        u0s = utility_func(x1,x2,alpha,beta)
        
    I = np.isclose(u0s, u0)
    x1 = x1[I]
    x2 = x2[I]

    I = (x1 >= 0) & (x2 >= 0)
    x1 = x1[I]
    x2 = x2[I]

    # e. sort
    I = np.argsort(x1)
    x1 = x1[I]
    x2 = x2[I]

    return x1,x2,u0


In [3]:
def plot_indifference_curves(preferences,alpha,beta):
    
    # a. create figure
    fig = plt.figure(figsize=(6,6),dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    # b. construct and plot indifference curves
    x1_max = 10
    x2_max = 10
    N = 100
    x0s = [(2,2),(4,4),(6,6)]
    for i, x0 in enumerate(x0s):

        if preferences == "saturated":
            
            u0 = -(x0[0]-alpha)**2 - (x0[1]-beta)**2
            radius = np.sqrt(-u0)
            circle = plt.Circle(
                (alpha, beta),
                radius,
                fill=False,
                color=colors[i],
            )
            ax.add_artist(circle)
            
        elif preferences == 'perfect_complements':
            
            u0 = np.fmin(alpha*x0[0],beta*x0[1])
            
            ax.plot([u0/alpha,u0/alpha],[u0/beta,x2_max],color=colors[i])
            ax.plot([u0/alpha,x1_max],[u0/beta,u0/beta],color=colors[i])
            
        else:
            
            x1,x2,u0 = find_indifference_curve(preferences,alpha,beta,x0,x1_max=x1_max,x2_max=x2_max,N=N)
            ax.plot(x1, x2, label=f"$u = {u0:.2f}$")
        
        ax.scatter(x0[0],x0[1])
        ax.set_xlabel('$x_1$')
        ax.set_ylabel('$x_2$')
        ax.set_xlim([0,x1_max])
        ax.set_ylim([0,x2_max])
    

# Interactive: Cobb-Douglas

$$
u(x_1,x_2) = x_1^{\alpha}x_2^{\beta}
$$

In [4]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('cobb_douglas',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.05,max=2.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.05,max=2.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=2.0, min=0.05, step=0.05), FloatSlid…

# Interactive: Constant Elasticity of Substitution (CES)

$$
u(x_1,x_2) = (\alpha x_1^{-\beta}+(1-\alpha)x_2^{-\beta})^{-1/\beta}
$$

In [5]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('ces',alpha,beta),
        alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.0, max=1.0,step=0.05,value=0.5),
        beta=widgets.FloatSlider(description=r'$\beta$',min=-0.95, max=20.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=1.0, step=0.05), FloatSlider(value=0…

# Interactive: Perfect substitutes

$$
u(x_1,x_2) = \alpha x_1 + \beta x_2
$$

In [6]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('perfect_substitutes',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=1.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=1.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=1.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Perfect complements

$$
u(x_1,x_2) = \min{\{\alpha x_1,\beta x_2}\}
$$

In [7]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('perfect_complements',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=1.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=1.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=1.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Quasi-linear (log)

$$
u(x_1,x_2) = \alpha\log(x_1) + \beta x_2
$$

In [8]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('quasi_log',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=5.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=5.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=5.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Quasi-linear (sqrt)

$$
u(x_1,x_2) = \alpha\sqrt{x_1} + \beta x_2
$$

In [9]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('quasi_sqrt',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=5.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=5.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=5.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Time-seperable with log

$$
u(x_1,x_2) = \log (x_1) + \beta \log(x_2)
$$

In [10]:
widgets.interact(lambda beta: plot_indifference_curves('additive_log',None,beta),
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=1.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\beta$', max=1.0, min=0.1, step=0.05), Output()), …

# Interactive: Time-seperable with CRRA

$$
u(x_1,x_2) = \frac{x_1^{1-\alpha}}{1-\alpha} + \beta \frac{x_2^{1-\alpha}}{1-\alpha}
$$

In [11]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('additive_crra',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=4.0,step=0.05,value=2.0), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=1.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=2.0, description='$\\alpha$', max=4.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Concave

$$
u(x_1,x_2) = \alpha x_1^2 + \beta x_2^2
$$

In [12]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('concave',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1,max=5.0,step=0.05,value=0.5), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1,max=5.0,step=0.05,value=0.5),
);

interactive(children=(FloatSlider(value=0.5, description='$\\alpha$', max=5.0, min=0.1, step=0.05), FloatSlide…

# Interactive: Saturated (non-montone)

$$
u(x_1,x_2) = -(x_1-\alpha)^2 - (x_2-\beta)^2
$$

In [13]:
widgets.interact(lambda alpha,beta: plot_indifference_curves('saturated',alpha,beta),
    alpha=widgets.FloatSlider(description=r'$\alpha$',min=0.1, max=10.0,step=0.05,value=5.0), 
    beta=widgets.FloatSlider(description=r'$\beta$',min=0.1, max=10.0,step=0.05,value=4.0),
);

interactive(children=(FloatSlider(value=5.0, description='$\\alpha$', max=10.0, min=0.1, step=0.05), FloatSlid…